In [57]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

In [58]:
# Load the dataset
data = pd.read_csv('/content/drive/MyDrive/Applicant-details.csv')

In [59]:
# Assuming you have a DataFrame or numpy array called 'data' containing your preprocessed data
X = data.drop('Loan_Default_Risk', axis=1).values
y = data['Loan_Default_Risk'].values

In [60]:
# Assuming 'data' is your DataFrame containing the data
data_encoded = pd.get_dummies(data)

In [61]:
# Split the data into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [62]:
from sklearn.preprocessing import OneHotEncoder

# Assuming 'data' is your DataFrame containing the preprocessed data
categorical_columns = ['Marital_Status', 'House_Ownership', 'Vehicle_Ownership(car)', 'Occupation', 'Residence_City', 'Residence_State']

# Perform one-hot encoding on the categorical columns
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
encoded_cols = encoder.fit_transform(data[categorical_columns])

# Drop the original categorical columns from the dataset
data = data.drop(categorical_columns, axis=1)

# Concatenate the encoded columns with the original dataset
data_encoded = np.concatenate([data.values, encoded_cols], axis=1)

# Split the data into X and y
X = data_encoded[:, :-1]
y = data_encoded[:, -1]

# Split the data into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Convert the data to float32
X_train = X_train.astype('float32')
X_val = X_val.astype('float32')
X_test = X_test.astype('float32')
y_train = y_train.astype('float32')
y_val = y_val.astype('float32')
y_test = y_test.astype('float32')

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [63]:
from sklearn.preprocessing import StandardScaler
# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

In [64]:
# Define neural network architecture
model = keras.Sequential([
    keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

In [65]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [66]:
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

Epoch 1/10
2188/2188 [==============================] - 12s 5ms/step - loss: 856.2120 - accuracy: 0.8457 - val_loss: 1.8068 - val_accuracy: 0.9061
Epoch 2/10
2188/2188 [==============================] - 16s 7ms/step - loss: 2.6406 - accuracy: 0.9045 - val_loss: 0.3116 - val_accuracy: 0.9061
Epoch 3/10
2188/2188 [==============================] - 8s 4ms/step - loss: 1.6060 - accuracy: 0.9075 - val_loss: 0.3117 - val_accuracy: 0.9061
Epoch 4/10
2188/2188 [==============================] - 8s 4ms/step - loss: 1.2057 - accuracy: 0.9078 - val_loss: 0.3116 - val_accuracy: 0.9061
Epoch 5/10
2188/2188 [==============================] - 8s 3ms/step - loss: 0.7105 - accuracy: 0.9079 - val_loss: 0.3116 - val_accuracy: 0.9061
Epoch 6/10
2188/2188 [==============================] - 8s 4ms/step - loss: 0.9966 - accuracy: 0.9079 - val_loss: 0.3117 - val_accuracy: 0.9061
Epoch 7/10
2188/2188 [==============================] - 8s 3ms/step - loss: 0.3447 - accuracy: 0.9079 - val_loss: 0.3116 - val_accur

In [67]:
# Evaluate the model
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)

469/469 [==============================] - 1s 1ms/step


In [68]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test_scaled, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')

469/469 [==============================] - 1s 2ms/step - loss: 0.3207 - accuracy: 0.9016
Loss: 0.3207089602947235, Accuracy: 0.9016000032424927
